In [ ]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
print("GPUs available:", physical_devices)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
from functools import partial
import math
import os

import numpy as np

from ocml.datasets import build_ds_from_numpy, tfds_from_sampler
from ocml.evaluate import check_LLC, log_metrics
from ocml.models import conventional_dense, spectral_dense
from ocml.plot import plot_preds_ood
from ocml.priors import uniform_tabular
from ocml.train import train, SH_KR, BCE

In [ ]:
from types import SimpleNamespace

def get_config(debug=False):
  config = SimpleNamespace(
    batch_size = 128,  # should be not too small to ensure diversity.
    domain = [-5, 5.],  # domain on which to sample points.
    scaling = False,
    maxiter = 4,  # very important on high dimensional dataset.
    margin = 0.05,  # very important !
    lbda = 100.,  # important but not as much as `margin`. Must be high for best results.
    k_coef_lip = 1.,  # no reason to change this.
    spectral_dense = True,  # Mandatory for orthogonal networks. 
    deterministic = False,  # Better with random learning rates.
    overshoot_boundary = False,
    conventional = False,  # Conventional training (i.e without hKR and Lipschitz constraint) for sanity check.
    widths = [512, 512, 512, 512],
    warmup_epochs=5,
    epochs_per_plot=20,
  )
  return config

In [ ]:
debug = "SANDBOX" in os.environ
config = get_config(debug)
train_kwargs = {
  'domain': config.domain,
  'deterministic': config.deterministic,
  'overshoot_boundary': True
}

In [ ]:
import plotly.io as pio
print("PLOTLY_RENDERER:", pio.renderers.default)
try:
  import os
  os.environ['WANDB_NOTEBOOK_NAME'] = 'run_tabular.ipynb'
  import wandb
  wandb.login()
  wandb_available = True
except ModuleNotFoundError as e:
  print(e)
  print("Wandb logs will be removed.")
  wandb_available = False
plot_wandb = wandb_available and not debug  # Set to False to de-activate Wandb.
if plot_wandb:  
  import wandb
  group = os.environ.get("WANDB_GROUP", "sandbox_tabular")
  wandb.init(project="oneclass", group=group, config=config.__dict__)
else:
  try:
    wandb.finish()
  except Exception as e:
    print(e)
    
train_kwargs['log_metrics_fn'] = partial(log_metrics, plot_wandb=plot_wandb)

In [ ]:
# Train model.
if config.conventional:
  model = conventional_dense(widths=config.widths, input_shape=(6,))
else:
  model = spectral_dense(widths=config.widths, input_shape=(6,),
                         k_coef_lip=config.k_coef_lip)

if config.conventional:
  loss_fn = BCE()
else:
  loss_fn = SH_KR(config.margin, config.lbda)

In [ ]:
import pandas as pd
from tensorflow.keras.utils import get_file

try:
  path = '/data/datasets/tabular/thyroid.mat'
  thyroid_path = get_file(path, origin='https://www.dropbox.com/s/bih0e15a0fukftb/thyroid.mat?dl=1')
  print(thyroid_path)
except:
  print('Error downloading')
  raise

In [ ]:
import numpy as np
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd

def load_matfile(file_path):
    mat = loadmat(file_path)
    mat = {k:v for k, v in mat.items() if k[0] != '_'}
    df = pd.DataFrame(np.concatenate([mat['X'],mat['y']], axis=1))
    df.rename({6:'label'},axis=1,inplace=True)
    return df
  
df = load_matfile(thyroid_path)
df

In [ ]:
df.describe()

In [ ]:
normality = df[df['label'] == 0.].drop('label', axis=1)
anomalies = df[df['label'] == 1.].drop('label', axis=1).to_numpy()
print(f"Normality={len(normality)} Anomaly={len(anomalies)}")

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test = train_test_split(normality.to_numpy(), train_size=1839)
print(f"Train Size={len(x_train)} Test Size={len(x_test)+len(anomalies)}")

In [ ]:
epoch_length = math.ceil(len(x_train) / config.batch_size)
print(f"Epoch Length={epoch_length}")

In [ ]:
import sklearn.preprocessing as preprocessing

if config.scaling:
  scaler = preprocessing.StandardScaler()
  x_train = scaler.fit_transform(x_train)
  x_test = scaler.transform(x_test)
  normality = scaler.transform(normality)
  anomalies = scaler.transform(anomalies)

In [ ]:
from sklearn.neighbors import KDTree
print('Building tree... ')
kdt = KDTree(normality, leaf_size=30, metric='euclidean')
print('Built ! Queries on going... ')
dists, indexes = kdt.query(anomalies, k=20, return_distance=True)
print('Distances of each anomaly to 20 nearest normal points')
pd.DataFrame(dists).describe()

In [ ]:
class Categorical:
  def __init__(self, num_classes):
    self.num_classes = max(num_classes, 2)
  def scale_bounds(self, bounds):
    pass
  def is_outlier(x):
    return np.full(x.shape[0], False, dtype=bool)
  def encode(x):
    if self.num_classes == 2:
      return x.copy()
    one_hot = np.eye(self.num_classes)[x]
    return one_hot
  def sample(self, batch_size):
    if self.num_classes == 2:
      return np.random.randint(2, size=batch_size)[:,np.newaxis]
    classes = np.random.randint(self.num_classes, size=batch_size)
    one_hot = np.eye(self.num_classes)[classes]
    return one_hot

class Gaussian:
  def __init__(self, mean, std, bounds):
    self.mean = mean
    self.std = std
    self.threshold = bounds
  def scale_bounds(self, bounds):
    self.threshold *= bounds
  def is_outlier(x):
    return np.abs(x) > self.threshold * self.std
  def encode(x):
    return (x - mean) / std
  def sample(self, batch_size):
    emp_std = self.threshold * self.std
    return np.random.normal(self.mean, emp_std, (batch_size,1))

class LogUniform:
  def __init__(self, min_v, max_v, shift):
    self.min_v = min_v
    self.max_v = max_v
    self.shift = shift
    self.bounds = 1.
  def scale_bounds(self, bounds):
    self.bounds *= bounds
  def is_outlier(x):
    return np.logical_or(x < self.min_v, x > self.max_v)
  def encode(x):
    return np.log(x + self.shift)
  def sample(self, batch_size):
    min_v, max_v = self.min_v * self.bounds, self.max_v * self.bounds
    return np.random.uniform(min_v, max_v, (batch_size,1))  # uniform in log space

class Sampler:
    def __init__(self, bounds, samplers=None):
      self.samplers = [] if samplers is None else samplers
      self.bounds = bounds
      self.shift = 0.1
    def scale_bounds(self, bounds):
      self.bounds *= bounds
      for sampler in self.samplers:
          sampler.scale_bounds(bounds)
    def add(self, sampler):
      self.samplers.append(sampler)
    def check_integrity(self, batch_size, batch_size_ref):
      assert self.sample(batch_size).shape == (batch_size,) + batch_size_ref
    def encode_numeric_zscore(self, df, df_source, df_train, name, mean=None, sd=None):
      if mean is None:
          mean = df_train[name].mean()
      if sd is None:
          sd = df_train[name].std()
      if sd == 0:
          sd = 1
      df[name] = (df_source[name] - mean) / sd
      bounds = max(df[name].max(), -df[name].min()) / 2
      sampler = Gaussian(0., 1., bounds)
      self.add(sampler)
    def encode_logscale(self, df, df_source, name):
      df[name] = np.log(df_source[name] + self.shift)
      min_v = df[name].min()
      max_v = df[name].max()
      sampler = LogUniform(min_v, max_v, self.shift)
      self.add(sampler)
    def encode_robust_zscore(self, df, df_source, df_train, name, median=None, mad=None):
      """Robust Z-Score for better robustness against outliers in train set."""
      if median is None:
        median = df_train[name].median()
      if mad is None:
        absolute_deviation = (df_train[name] - median).abs()
        mad = absolute_deviation.median()
      if mad == 0:
        mad = 1
      df[name] = (df_source[name] - median) / mad * 0.6745
      bounds = max(df[name].max(), -df[name].min()) / 2
      sampler = Gaussian(0., 1., bounds)
      self.add(sampler)
    def encode_text_dummy(self, df, df_source, name):
      uniques = df_source[name].nunique()
      if uniques == 1:
        dummy_name = f"{name}-{df_source[name].iloc[0]}"
        df[dummy_name] = 1.
      elif uniques <= 2:
        dummy_name = f"is-{name}"
        dummies = pd.get_dummies(df_source[name], drop_first=True)
        df[dummy_name] = dummies[list(dummies.columns)[0]]
      else:  # No sparse when more than 1 class to ensure same distance between everyone
        dummies = pd.get_dummies(df_source[name])
        for x in dummies.columns:
          dummy_name = f"{name}-{x}"
          df[dummy_name] = dummies[x]
      sampler = Categorical(uniques)
      self.add(sampler)
    def fit_transform(self, df_source, df_train, continuous_policy, discrete_cols=[]):
      assert continuous_policy in ['robust', 'logscale', 'zscore']
      cols = list(df_source.columns)
      df = pd.DataFrame(index=df_source.index)
      for col in cols:
        if col == 'label':
          continue
        if col in discrete_cols:
          self.encode_text_dummy(df, df_source, col)
        elif continuous_policy == 'robust':
          self.encode_robust_zscore(df, df_source, df_train, col)
        elif continuous_policy == 'logscale':
          self.encode_logscale(df, df_source, col)
        elif continuous_policy == 'zscore':
          self.encode_numeric_zscore(df, df_source, df_train, col)
      df['label'] = df_source['label'].copy()
      self.check_integrity(16, (df.shape[1]-1,))
      return df
    def sample(self, batch_size):
      samples = [sampler.sample(batch_size) for sampler in self.samplers]
      samples = np.concatenate(samples, axis=1)
      return samples

In [ ]:
sampler = Sampler(bounds = 5)
continuous_policy = 'zscore'
dt = sampler.fit_transform(df, normality, continuous_policy=continuous_policy)

In [ ]:
dt[dt['label'] == 0.].describe()

In [ ]:
dt[dt['label'] == 1.].describe()

In [ ]:
adv = pd.DataFrame(sampler.sample(10))
adv.columns = dt.columns[:-1]
adv

In [ ]:
dists, indexes = kdt.query(adv.to_numpy(), k=20, return_distance=True)
pd.DataFrame(dists).describe()

In [ ]:
from sklearn.metrics import roc_auc_score, f1_score
import scipy

def compute_precision_recall(ytest, yanomalies, T):
  pred_test = ytest >= T
  pred_anomalies = yanomalies < T
  tp = pred_test.sum()
  tn = pred_anomalies.sum()
  fp = len(yanomalies) - tn
  fn = len(ytest) - tp
  recall_in = tp / (tp + fn + 1e-8) * 100
  recall_out = tn / (tn + fp + 1e-8) * 100
  precision_in = tp / (tp + fp + 1e-8) * 100
  precision_out = tn / (tn + fn + 1e-8) * 100
  preds = pred_test, pred_anomalies
  precision_recall = (recall_in, recall_out, precision_in, precision_out)
  return preds, precision_recall, T

def seek_equal_precision_recall(ytest, yanomalies):
  """Apply protocole of TQM and HRN to make precision=recall=F1."""
  a = min(ytest.min(), yanomalies.min())
  b = min(ytest.max(), yanomalies.max())
  def fun(T):
    preds, precision_recall, T = compute_precision_recall(ytest, yanomalies, T)
    recall_in, _, precision_in, _ = precision_recall
    return recall_in - precision_in
  T = scipy.optimize.bisect(fun, a, b)
  return T

def evaluate(epoch, model, xtest, anomalies, threshold=None):
    test_size, anomalies_size = len(xtest), len(anomalies)
    xx = np.concatenate([xtest, anomalies], axis=0)
    _ = model(anomalies, training=True) # garbage in to apply bjorck.
    yy = model.predict(xx, verbose=1, batch_size=2048).flatten()
    ytest, yanomalies = np.split(yy, indices_or_sections=[test_size])
    mean_in, std_in = ytest.mean(), ytest.std()
    mean_out, std_out = yanomalies.mean(), yanomalies.std()
    if threshold is None:
       threshold = seek_equal_precision_recall(ytest, yanomalies)
    preds, precision_recall, threshold = compute_precision_recall(ytest, yanomalies, threshold)
    pred_test, pred_anomalies = preds
    recall_in, recall_out, precision_in, precision_out = precision_recall
    true_labels = np.concatenate([np.ones(test_size), np.zeros(anomalies_size)], axis=0)
    roc_auc = roc_auc_score(true_labels, yy)
    pred_yy = np.concatenate([pred_test, 1-pred_anomalies], axis=0)
    f1 = f1_score(true_labels, pred_yy) * 100
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    percentiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    trainstats = pd.DataFrame(ytest).describe(percentiles).transpose()
    teststats = pd.DataFrame(yanomalies).describe(percentiles).transpose()
    stats = pd.concat([trainstats,teststats], ignore_index=True).round(4)
    stats.index = ['test','anomalies']
    if (epoch+1)%5== 0:
        print(stats)
    margin_T = (threshold - config.margin) / config.margin * 100
    print(f"Mean-In={mean_in:.2f}±{std_in:.2f} Mean-Out={mean_out:.2f}±{std_out:.2f} T={threshold:.4f} TMarginError={margin_T:.2f}%")
    print(f"False-Alarm={100-recall_in:.2f}% Sensivity-Anomalies={recall_out:.2f} Precision-Anomaly={precision_out:.2f}%")
    print(f"[IMPORTANT] ROC_AUC={roc_auc:.2f} Precision={precision_in:.2f}% Recall={recall_in:.2f}% F1={f1:.2f}")
    if plot_wandb:
      wandb.log({'roc_auc': roc_auc, 'recall':recall_in, 'precision':precision_in, 'f1':f1, 'T':threshold, 'TMarginError':margin_T})

In [ ]:
# Create positive examples dataset.
p_dataset = build_ds_from_numpy(x_train, config.batch_size)

In [ ]:
# Create optimizer class.
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0005)

# Initialize the network.
gen = tf.random.Generator.from_seed(4321)  # reproducible sampling.
p_batch = next(iter(p_dataset))
_ = model(p_batch, training=True)  # dummy forward to trigger initialization.
model.summary()

In [ ]:
# Adversarial distribution.
def sampler_fn(gen, batch_size, input_shape):
  del gen  # unused.
  del input_shape  # unused.
  return sampler.sample(batch_size)

q_dataset = tfds_from_sampler(sampler_fn, gen, config.batch_size, p_batch.shape[1:])
Q0 = next(iter(q_dataset))

In [ ]:
num_epochs = config.warmup_epochs
for epoch in range(num_epochs):
  train(model, opt, loss_fn, gen, p_dataset, q_dataset, epoch_length, maxiter=0, **train_kwargs)
  evaluate(epoch, model, x_test, anomalies)
plot_preds_ood(epoch, model, tf.constant(x_train), tf.constant(x_test), tf.constant(anomalies), plot_histogram=True, plot_wandb=False)

In [ ]:
for epoch in range(config.epochs_per_plot):
  train(model, opt, loss_fn, gen, p_dataset, q_dataset, epoch_length, maxiter=config.maxiter, **train_kwargs)
  evaluate(epoch, model, x_test, anomalies)
plot_preds_ood(epoch, model, tf.constant(x_train), tf.constant(x_test), tf.constant(anomalies), plot_histogram=True, plot_wandb=plot_wandb)

In [ ]:
for epoch in range(config.epochs_per_plot):
  train(model, opt, loss_fn, gen, p_dataset, q_dataset, epoch_length, maxiter=config.maxiter, **train_kwargs)
  evaluate(epoch, model, x_test, anomalies)
plot_preds_ood(epoch, model, tf.constant(x_train), tf.constant(x_test), tf.constant(anomalies), plot_histogram=True, plot_wandb=plot_wandb)

In [ ]:
if plot_wandb:
  wandb.finish()